<a href="https://colab.research.google.com/github/SouparnaChatterjee/Meta-AIML/blob/main/model2(finetuning%20existing%20models%20hugging%20face).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install flask sentence-transformers
!pip install -q flask-cors  # For handling CORS issues in Colab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 960.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:

!pip install pyngrok

In [3]:
import os
from pyngrok import ngrok, conf

In [4]:
# Import libraries and load model
import json
import torch
from flask import Flask, request, jsonify, render_template_string
from sentence_transformers import SentenceTransformer, util

# Load Hugging Face model for intent detection
print("Loading model from Hugging Face...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded!")

# Define intents and their example phrases
intent_references = {
    "services": [
        "What services do you offer?",
        "Tell me about your services",
        "What can you help me with?",
        "What services are available?",
        "What does your company do?"
    ],
    "location": [
        "Where is your office?",
        "What's your address?",
        "How can I find you?",
        "Where are you located?",
        "What's your location?"
    ],
    "portfolio": [
        "Show me your portfolio",
        "Can I see your work?",
        "Show me previous projects",
        "What have you worked on before?",
        "Examples of your work"
    ],
    "contact": [
        "How can I contact you?",
        "What's your phone number?",
        "How do I reach you?",
        "Contact information",
        "Get in touch"
    ],
    "lead": [
        "I'm interested in your services",
        "I want more information",
        "I'd like to discuss a project",
        "Can I get a quote?",
        "I want to hire you"
    ]
}

# Pre-compute embeddings for all reference phrases
intent_embeddings = {}
for intent, phrases in intent_references.items():
    intent_embeddings[intent] = model.encode(phrases)

# Define responses for each intent
responses = {
    "services": {
        "type": "text",
        "content": "We offer the following services:",
        "options": [
            "Business Consulting",
            "Tax Planning",
            "Audit Services",
            "Financial Advisory",
            "Compliance Services"
        ]
    },
    "location": {
        "type": "text",
        "content": "Our office is located at: 123 Business Park, Suite 456, Example City. Here's a map:",
        "map": True
    },
    "portfolio": {
        "type": "redirect",
        "content": "Let me show you our portfolio of work.",
        "redirect": "#portfolio-section"
    },
    "contact": {
        "type": "text",
        "content": "You can reach us through:",
        "options": [
            "Phone: (123) 456-7890",
            "Email: contact@llpproject.com",
            "Schedule a Call"
        ]
    },
    "lead": {
        "type": "form",
        "content": "Please provide your information to receive more details:",
        "form_fields": ["Name", "Email", "Phone", "Service Interested In"]
    },
    "default": {
        "type": "options",
        "content": "I'm not sure what you're looking for. Please select an option:",
        "options": ["Services", "Portfolio", "Contact Us", "Location", "Request Information"]
    }
}

Loading model from Hugging Face...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded!


In [5]:
# Function to detect intent using the model
def detect_intent(query):
    # Encode the user query
    query_embedding = model.encode(query)

    # Find the best matching intent
    best_intent = None
    best_score = -1

    for intent, embeddings in intent_embeddings.items():
        # Calculate similarity with each reference phrase
        similarities = util.cos_sim(query_embedding, embeddings)
        max_similarity = torch.max(similarities).item()

        if max_similarity > best_score:
            best_score = max_similarity
            best_intent = intent

    # Use a threshold to determine if we have a confident match
    if best_score > 0.6:
        return best_intent, best_score
    else:
        return "default", best_score

# Function to handle user messages
def process_message(message):
    intent, confidence = detect_intent(message)
    print(f"Detected intent: {intent} (confidence: {confidence:.2f})")
    return responses[intent]

# Function to handle option selection
def process_option(option):
    # First try to match directly to an intent based on option text
    option_lower = option.lower()

    if "service" in option_lower:
        return responses["services"]
    elif "portfolio" in option_lower:
        return responses["portfolio"]
    elif "contact" in option_lower:
        return responses["contact"]
    elif "location" in option_lower:
        return responses["location"]
    elif "information" in option_lower or "request" in option_lower:
        return responses["lead"]

    # If no direct match, use the NLU model
    return process_message(option)# Function to detect intent using the model
def detect_intent(query):
    # Encode the user query
    query_embedding = model.encode(query)

    # Find the best matching intent
    best_intent = None
    best_score = -1

    for intent, embeddings in intent_embeddings.items():
        # Calculate similarity with each reference phrase
        similarities = util.cos_sim(query_embedding, embeddings)
        max_similarity = torch.max(similarities).item()

        if max_similarity > best_score:
            best_score = max_similarity
            best_intent = intent

    # Use a threshold to determine if we have a confident match
    if best_score > 0.6:
        return best_intent, best_score
    else:
        return "default", best_score

# Function to handle user messages
def process_message(message):
    intent, confidence = detect_intent(message)
    print(f"Detected intent: {intent} (confidence: {confidence:.2f})")
    return responses[intent]

# Function to handle option selection
def process_option(option):
    # First try to match directly to an intent based on option text
    option_lower = option.lower()

    if "service" in option_lower:
        return responses["services"]
    elif "portfolio" in option_lower:
        return responses["portfolio"]
    elif "contact" in option_lower:
        return responses["contact"]
    elif "location" in option_lower:
        return responses["location"]
    elif "information" in option_lower or "request" in option_lower:
        return responses["lead"]

    # If no direct match, use the NLU model
    return process_message(option)

In [7]:
# Create Flask app
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS  # Import CORS

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# HTML template for chatbot interface
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html>
<head>
    <title>LLP Project Chatbot</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
        }
        .container {
            max-width: 500px;
            margin: 20px auto;
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 0 10px rgba(0,0,0,0.1);
        }
        .header {
            background-color: #4a69bd;
            color: white;
            padding: 15px;
            text-align: center;
            font-size: 18px;
            font-weight: bold;
        }
        .chat-container {
            height: 400px;
            overflow-y: auto;
            padding: 15px;
            background-color: white;
        }
        .message {
            margin-bottom: 10px;
            padding: 8px 15px;
            border-radius: 20px;
            max-width: 70%;
            word-wrap: break-word;
        }
        .bot-message {
            background-color: #e9f5ff;
            color: #333;
            margin-right: auto;
            border-bottom-left-radius: 5px;
        }
        .user-message {
            background-color: #4a69bd;
            color: white;
            margin-left: auto;
            border-bottom-right-radius: 5px;
        }
        .input-container {
            display: flex;
            padding: 10px;
            background-color: #f9f9f9;
            border-top: 1px solid #ddd;
        }
        #user-input {
            flex: 1;
            padding: 10px;
            border: 1px solid #ddd;
            border-radius: 20px;
            outline: none;
        }
        #send-button {
            background-color: #4a69bd;
            color: white;
            border: none;
            padding: 10px 15px;
            margin-left: 10px;
            border-radius: 20px;
            cursor: pointer;
        }
        .options-container {
            display: flex;
            flex-wrap: wrap;
            gap: 5px;
            margin-top: 10px;
        }
        .option-button {
            background-color: #e9f5ff;
            border: 1px solid #4a69bd;
            color: #4a69bd;
            padding: 8px 15px;
            border-radius: 15px;
            cursor: pointer;
            font-size: 14px;
        }
        .option-button:hover {
            background-color: #d0e6ff;
        }
        .form-container {
            background-color: #e9f5ff;
            padding: 10px;
            border-radius: 10px;
            margin-top: 10px;
        }
        .form-field {
            margin-bottom: 10px;
        }
        .form-field input {
            width: 100%;
            padding: 8px;
            border: 1px solid #ddd;
            border-radius: 5px;
        }
        .submit-form {
            background-color: #4a69bd;
            color: white;
            border: none;
            padding: 8px 15px;
            border-radius: 5px;
            cursor: pointer;
        }
        .map-container {
            background-color: #e9f5ff;
            padding: 10px;
            border-radius: 10px;
            margin-top: 10px;
            text-align: center;
        }
        .map-placeholder {
            width: 100%;
            height: 150px;
            background-color: #ddd;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #666;
            font-style: italic;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">LLP Project Assistant</div>
        <div class="chat-container" id="chat-container">
            <!-- Chat messages will appear here -->
        </div>
        <div class="input-container">
            <input type="text" id="user-input" placeholder="Type your message...">
            <button id="send-button">Send</button>
        </div>
    </div>

    <script>
        // Base URL for API calls - will be set dynamically
        const API_BASE_URL = window.location.href;

        document.addEventListener('DOMContentLoaded', function() {
            // Add initial bot message
            addBotMessage("Hello! Welcome to LLP Project. How can I help you today?");

            // Display initial options
            displayOptions([
                "What services do you offer?",
                "Show me your portfolio",
                "How can I contact you?",
                "Where are you located?",
                "I'd like more information"
            ]);

            // Send button click event
            document.getElementById('send-button').addEventListener('click', sendMessage);

            // Enter key press event
            document.getElementById('user-input').addEventListener('keypress', function(e) {
                if (e.key === 'Enter') {
                    sendMessage();
                }
            });
        });

        function sendMessage() {
            const userInput = document.getElementById('user-input').value.trim();
            if (userInput === '') return;

            // Display user message
            addUserMessage(userInput);

            // Clear input field
            document.getElementById('user-input').value = '';

            // Send to backend and get response
            fetch(API_BASE_URL + 'api/chat', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ message: userInput }),
            })
            .then(response => response.json())
            .then(data => {
                handleBotResponse(data);
            })
            .catch(error => {
                console.error('Error:', error);
                addBotMessage("Sorry, there was an error processing your request.");
            });
        }

        function handleOptionClick(option) {
            addUserMessage(option);

            fetch(API_BASE_URL + 'api/option', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json',
                },
                body: JSON.stringify({ option: option }),
            })
            .then(response => response.json())
            .then(data => {
                handleBotResponse(data);
            })
            .catch(error => {
                console.error('Error:', error);
                addBotMessage("Sorry, there was an error processing your selection.");
            });
        }

        function handleBotResponse(response) {
            addBotMessage(response.content);

            if (response.options && response.options.length > 0) {
                displayOptions(response.options);
            }

            if (response.map) {
                displayMap();
            }

            if (response.form_fields && response.form_fields.length > 0) {
                displayForm(response.form_fields);
            }

            if (response.redirect) {
                addBotMessage("Redirecting you to the requested page...");
                setTimeout(() => {
                    alert("In a real implementation, this would redirect to: " + response.redirect);
                }, 1000);
            }
        }

        function addUserMessage(message) {
            const chatContainer = document.getElementById('chat-container');
            const messageElement = document.createElement('div');
            messageElement.classList.add('message', 'user-message');
            messageElement.textContent = message;
            chatContainer.appendChild(messageElement);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function addBotMessage(message) {
            const chatContainer = document.getElementById('chat-container');
            const messageElement = document.createElement('div');
            messageElement.classList.add('message', 'bot-message');
            messageElement.textContent = message;
            chatContainer.appendChild(messageElement);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function displayOptions(options) {
            const chatContainer = document.getElementById('chat-container');
            const optionsContainer = document.createElement('div');
            optionsContainer.classList.add('options-container');

            options.forEach(option => {
                const button = document.createElement('button');
                button.classList.add('option-button');
                button.textContent = option;
                button.addEventListener('click', function() {
                    handleOptionClick(option);
                });
                optionsContainer.appendChild(button);
            });

            chatContainer.appendChild(optionsContainer);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function displayMap() {
            const chatContainer = document.getElementById('chat-container');
            const mapContainer = document.createElement('div');
            mapContainer.classList.add('map-container');

            const mapPlaceholder = document.createElement('div');
            mapPlaceholder.classList.add('map-placeholder');
            mapPlaceholder.textContent = "Map would be displayed here";

            mapContainer.appendChild(mapPlaceholder);
            chatContainer.appendChild(mapContainer);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }

        function displayForm(fields) {
            const chatContainer = document.getElementById('chat-container');
            const formContainer = document.createElement('div');
            formContainer.classList.add('form-container');

            fields.forEach(field => {
                const formField = document.createElement('div');
                formField.classList.add('form-field');

                const label = document.createElement('label');
                label.textContent = field + ":";

                const input = document.createElement('input');
                input.type = field.toLowerCase().includes('email') ? 'email' : 'text';
                input.placeholder = "Enter your " + field.toLowerCase();

                formField.appendChild(label);
                formField.appendChild(input);
                formContainer.appendChild(formField);
            });

            const submitButton = document.createElement('button');
            submitButton.classList.add('submit-form');
            submitButton.textContent = "Submit";
            submitButton.addEventListener('click', function() {
                const inputs = formContainer.querySelectorAll('input');
                let formData = {};
                let allFilled = true;

                inputs.forEach(input => {
                    if (input.value.trim() === '') {
                        allFilled = false;
                    }
                    formData[input.placeholder.replace('Enter your ', '')] = input.value;
                });

                if (allFilled) {
                    addBotMessage("Thank you for your information! Our team will contact you soon.");
                    formContainer.style.display = 'none';

                    // In a real implementation, you would send this data to a server
                    console.log("Form data:", formData);
                } else {
                    addBotMessage("Please fill in all fields.");
                }
            });

            formContainer.appendChild(submitButton);
            chatContainer.appendChild(formContainer);
            chatContainer.scrollTop = chatContainer.scrollHeight;
        }
    </script>
</body>
</html>
'''

# Define API routes
@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get('message', '')
    response = process_message(user_message)
    return jsonify(response)

@app.route('/api/option', methods=['POST'])
def option():
    data = request.json
    option_selected = data.get('option', '')
    response = process_option(option_selected)
    return jsonify(response)

In [8]:
# Install required packages
!pip install gradio sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.1 MB/s eta 0:00:00


In [9]:
# Import libraries
import gradio as gr
import torch
from sentence_transformers import SentenceTransformer, util

# Load Hugging Face model for intent detection
print("Loading model from Hugging Face...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded!")

# Define intents and their example phrases
intent_references = {
    "services": [
        "What services do you offer?",
        "Tell me about your services",
        "What can you help me with?",
        "What services are available?",
        "What does your company do?"
    ],
    "location": [
        "Where is your office?",
        "What's your address?",
        "How can I find you?",
        "Where are you located?",
        "What's your location?"
    ],
    "portfolio": [
        "Show me your portfolio",
        "Can I see your work?",
        "Show me previous projects",
        "What have you worked on before?",
        "Examples of your work"
    ],
    "contact": [
        "How can I contact you?",
        "What's your phone number?",
        "How do I reach you?",
        "Contact information",
        "Get in touch"
    ],
    "lead": [
        "I'm interested in your services",
        "I want more information",
        "I'd like to discuss a project",
        "Can I get a quote?",
        "I want to hire you"
    ]
}

# Pre-compute embeddings for all reference phrases
intent_embeddings = {}
for intent, phrases in intent_references.items():
    intent_embeddings[intent] = model.encode(phrases)

# Define responses for each intent
responses = {
    "services": "We offer the following services:\n- Business Consulting\n- Tax Planning\n- Audit Services\n- Financial Advisory\n- Compliance Services\n\nWhich service are you interested in?",
    "location": "Our office is located at: 123 Business Park, Suite 456, Example City. You can find us on Google Maps by searching 'LLP Project Office'.",
    "portfolio": "Our portfolio includes projects for clients in various industries. You can view our complete portfolio on our website at www.llpproject.com/portfolio",
    "contact": "You can reach us through:\n- Phone: (123) 456-7890\n- Email: contact@llpproject.com\n- Social Media: @LLPProject",
    "lead": "I'd be happy to collect your information for our team to contact you. Please provide:\n1. Your Name\n2. Email Address\n3. Phone Number\n4. Service you're interested in",
    "default": "I'm not sure what you're looking for. You can ask me about our services, location, portfolio, or contact information. How can I help you today?"
}

# Function to detect intent using the model
def detect_intent(query):
    # Encode the user query
    query_embedding = model.encode(query)

    # Find the best matching intent
    best_intent = None
    best_score = -1

    for intent, embeddings in intent_embeddings.items():
        # Calculate similarity with each reference phrase
        similarities = util.cos_sim(query_embedding, embeddings)
        max_similarity = torch.max(similarities).item()

        if max_similarity > best_score:
            best_score = max_similarity
            best_intent = intent

    # Use a threshold to determine if we have a confident match
    if best_score > 0.6:
        return best_intent, best_score
    else:
        return "default", best_score

Loading model from Hugging Face...
Model loaded!


In [12]:
# Import libraries
import gradio as gr
import torch
from sentence_transformers import SentenceTransformer, util

# Load Hugging Face model for intent detection
print("Loading model from Hugging Face...")
model = SentenceTransformer('all-MiniLM-L6-v2')
print("Model loaded!")

# Define intents and their example phrases
intent_references = {
    "services": [
        "What services do you offer?",
        "Tell me about your services",
        "What can you help me with?",
        "What services are available?",
        "What does your company do?"
    ],
    "location": [
        "Where is your office?",
        "What's your address?",
        "How can I find you?",
        "Where are you located?",
        "What's your location?"
    ],
    "portfolio": [
        "Show me your portfolio",
        "Can I see your work?",
        "Show me previous projects",
        "What have you worked on before?",
        "Examples of your work"
    ],
    "contact": [
        "How can I contact you?",
        "What's your phone number?",
        "How do I reach you?",
        "Contact information",
        "Get in touch"
    ],
    "lead": [
        "I'm interested in your services",
        "I want more information",
        "I'd like to discuss a project",
        "Can I get a quote?",
        "I want to hire you"
    ]
}

# Pre-compute embeddings for all reference phrases
intent_embeddings = {}
for intent, phrases in intent_references.items():
    intent_embeddings[intent] = model.encode(phrases)

# Define responses for each intent
responses = {
    "services": "We offer the following services:\n- Business Consulting\n- Tax Planning\n- Audit Services\n- Financial Advisory\n- Compliance Services\n\nWhich service are you interested in?",
    "location": "Our office is located at: 123 Business Park, Suite 456, Example City. You can find us on Google Maps by searching 'LLP Project Office'.",
    "portfolio": "Our portfolio includes projects for clients in various industries. You can view our complete portfolio on our website at www.llpproject.com/portfolio",
    "contact": "You can reach us through:\n- Phone: (123) 456-7890\n- Email: contact@llpproject.com\n- Social Media: @LLPProject",
    "lead": "I'd be happy to collect your information for our team to contact you. Please provide:\n1. Your Name\n2. Email Address\n3. Phone Number\n4. Service you're interested in",
    "default": "I'm not sure what you're looking for. You can ask me about our services, location, portfolio, or contact information. How can I help you today?"
}

# Function to detect intent using the model
def detect_intent(query):
    # Encode the user query
    query_embedding = model.encode(query)

    # Find the best matching intent
    best_intent = None
    best_score = -1

    for intent, embeddings in intent_embeddings.items():
        # Calculate similarity with each reference phrase
        similarities = util.cos_sim(query_embedding, embeddings)
        max_similarity = torch.max(similarities).item()

        if max_similarity > best_score:
            best_score = max_similarity
            best_intent = intent

    # Use a threshold to determine if we have a confident match
    if best_score > 0.6:
        return best_intent, best_score
    else:
        return "default", best_score

# Function to process messages and generate responses
def chat(message, history):
    # Get intent and confidence score
    intent, confidence = detect_intent(message)
    print(f"Detected intent: {intent} (confidence: {confidence:.2f}) for message: {message}")

    # Return appropriate response
    return responses[intent]

# Create Gradio chat interface
demo = gr.ChatInterface(
    fn=chat,
    title="LLP Project Chatbot",
    description="Ask about our services, location, portfolio, or contact information.",
    examples=["What services do you offer?", "Where is your office located?", "Show me your portfolio", "How can I contact you?", "I'm interested in your services"]
)

# Launch the interface with public URL
print("Starting the chatbot... This may take a moment.")
demo.launch(share=True)

Loading model from Hugging Face...
Model loaded!


/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:338: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Starting the chatbot... This may take a moment.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7cecb21d416b9f43a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
